In [1]:
import os
import numpy as np
import importlib
import json
from datetime import datetime

# PyTorch
import torch
from torch.utils.data import DataLoader

# Utils
import utils
importlib.reload(utils)

from utils import get_files, save_image, make_dirs, get_model, select_gpu, get_data, get_class_weigths
from utils import CustomTransforms, My_data, FocalLoss

import ProjectedGradientDescent
importlib.reload(ProjectedGradientDescent)

<module 'ProjectedGradientDescent' from '/home/jovyan/ProjectedGradientDescent.py'>

In [2]:
# Set a higher max split size to avoid memory problems
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [3]:
device = torch.device(f"cuda:{select_gpu()}" if torch.cuda.is_available() else "cpu")

print(device)

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(torch.cuda.memory_summary(device=None, abbreviated=False))
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False

Selecting GPU 0 with 14903MB free memory
cuda:0
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from lar

In [4]:
custom_transforms = CustomTransforms()
test_transform = custom_transforms.get_transform("test")

In [5]:
# Load train and test files that are used for the model.
train_dict, test_dict = get_data(os.path.join(os.getcwd(), "BreaKHis_v1"))

Opening /home/jovyan/txt/train.txt.txt
Opening /home/jovyan/txt/test.txt.txt


In [6]:
train_files = "./dataset/train/original/**/**/*.png"

In [7]:
org_norm = My_data(get_files(train_files), transforms=test_transform)
org_dataloader = DataLoader(org_norm)

In [8]:
model_name = "resnet"
model = get_model(device, model_name)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

In [9]:
# Avoid using with torch.no_grad()
op_advs = ProjectedGradientDescent.pgd_attack(org_dataloader, model, device, get_class_weigths(train_dict).to(device))

In [ ]:
make_dirs("train/pgd_attack/"+model_name+"/")
with torch.no_grad():
    for i, (image, label) in enumerate(org_dataloader):
        path = org_norm.__getpath__(i)
        save_image(op_advs[i][0], "./dataset/train/pgd_attack/"+model_name+"/" + path)
